# Builder Tutorial number 10

The builder tutorials demonstrate how to build an operational GSFLOW model using `pyGSFLOW` from shapefile, DEM, and other common data sources. These tutorials focus on the `gsflow.builder` classes.

## Working with climate information

In this tutorial, we give an overview of how to translate climate information to PRMS parameters. The methods outlined here use raster resampling methods outlined in `Builder_tutorial_2` and use pandas dataframes to create climate information. Two different climate representation methods are presented

In [1]:
import os
import shapefile
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import pandas as pd
import flopy
from gsflow.builder import builder_utils as bu
import gsflow

# silence pandas setting with copy warning
pd.options.mode.chained_assignment = None

## Applying temp_1sta module parameters to the Sagehen 50m example problem

The temp_1sta module in PRMS allows the user to define their climate using a single climate station and adjustment factors based on lapse rates and aspect. In this example the methods are applied directly to the Sagehen 50m model as they are presented.

Let's start by defining paths and loading the PRMS parameter file from the previous tutorial and the MODFLOW model

In [2]:
# define the input and output data paths
input_ws = os.path.join("data", "sagehen", "50m_tutorials")
geospatial_ws = os.path.join("data", "geospatial")
output_ws = os.path.join("data", "temp")

# Set modflow model and the prms parameter file paths
modflow_nam = "sagehen_50m.nam"
parameter_file = os.path.join(input_ws, "sagehen_50m_lu_soil.params")

# set the pour point shapefile path
shp_file = os.path.join(geospatial_ws, "model_points.shp")

Load the MODFLOW model and PRMS parameter file

In [3]:
ml = gsflow.modflow.Modflow.load(modflow_nam, model_ws=input_ws)
parameters = gsflow.prms.PrmsParameters.load_from_file(parameter_file)

# check the modelgrid coordinate information to make sure it loaded properly
print(ml.modelgrid.xoffset, ml.modelgrid.yoffset)

   loading iuzfbnd array...
   loading irunbnd array...
   loading vks array...
   loading eps array...
   loading thts array...
stress period 1:
   loading finf array...
stress period 2:
------------------------------------
Reading parameter file : sagehen_50m_lu_soil.params
------------------------------------
214270.0 4366610.0


### Define paths to climate information

In this example we use both climate station information from Sagehen Creek co-operative station (University of California, Berkely, 2008) and PRISM 30-year normals (PRISM Climate Group, 2022) rasters to define the climate.

Let's start by defining paths for the climate station information.

In [4]:
climate_ws = os.path.join(geospatial_ws, "climate")

climate_sta_file = os.path.join(climate_ws, "sagehen_climate.csv")
lapse_rate_file = os.path.join(climate_ws, "sagehen_lapse_rates.csv")

Now let's programatically define the paths for the monthly PRISM 30 year normals raster data

In [5]:
prism = {"ppt_utm": [], "tmax_utm": [], "tmin_utm": []}
for folder in prism.keys():
    for f in os.listdir(os.path.join(climate_ws, folder)):
        if os.path.isfile(os.path.join(climate_ws, folder, f)) and f.endswith(".img"):
            prism[folder].append(os.path.join(climate_ws, folder, f))

# inspect ppt to make sure we've collected all of the precip filenames
for f in prism["ppt_utm"]:
    print(os.path.split(f)[-1])

PRISM_ppt_30yr_normal_800mM2_01_bil.img
PRISM_ppt_30yr_normal_800mM2_02_bil.img
PRISM_ppt_30yr_normal_800mM2_03_bil.img
PRISM_ppt_30yr_normal_800mM2_04_bil.img
PRISM_ppt_30yr_normal_800mM2_05_bil.img
PRISM_ppt_30yr_normal_800mM2_06_bil.img
PRISM_ppt_30yr_normal_800mM2_07_bil.img
PRISM_ppt_30yr_normal_800mM2_08_bil.img
PRISM_ppt_30yr_normal_800mM2_09_bil.img
PRISM_ppt_30yr_normal_800mM2_10_bil.img
PRISM_ppt_30yr_normal_800mM2_11_bil.img
PRISM_ppt_30yr_normal_800mM2_12_bil.img


### Load the climate station information

The climate station information is stored in a csv file. This allows it to be easily loaded in python using pandas

In [6]:
climate_df = pd.read_csv(climate_sta_file)
lapse_df = pd.read_csv(lapse_rate_file)

climate_df.head()

,tmax,tmin,precip,runoff,date
0,85,30,0.0,2.3,10/1/1980
1,81,32,0.0,2.3,10/2/1980
2,83,30,0.0,2.3,10/3/1980
3,80,32,0.0,2.3,10/4/1980
4,80,32,0.0,2.3,10/5/1980


Runoff information from the USGS gage 10343500 SAGEHEN C NR TRUCKEE CA is also stored in the climate station information file

### Loading the climate Raster Data

Loading and resampling of raster data is performed using the Raster resampling methods outlined in Builder tutorial 2. Because this is a computationally intensive process, code is provided for completeness, however the default behavior of this notebook is to skip this step and load the saved ASCII versions. Change `resample_rasters=False` to `resample_rasters=True` to run the raster sampling.

In [7]:
resample_rasters = False

In [8]:
# loop over all the datatypes and rasters to create a single yearly resampled file with monthly values entries
if resample_rasters:
    rs_files = {
        "ppt_utm": os.path.join(output_ws, "ppt_50m.txt"),
        "tmax_utm": os.path.join(output_ws, "tmax_50m.txt"),
        "tmin_utm": os.path.join(output_ws, "tmin_50m.txt")
    }
    
    # create a loop to reduce code clutter
    for ctype, raster_list in prism.items():
        output = []
        rs_file = rs_files[ctype]
        for raster_file in raster_list:
            raster = flopy.utils.Raster.load(raster_file)
            array = raster.resample_to_grid(
                ml.modelgrid,
                band=raster.bands[0],
                method="linear",
            )
            output.append(array.ravel())
        output = np.array(output)
        print(os.path.split(rs_file)[-1])
        np.savetxt(rs_file, output)

### Loading the resampled raster data

Resampled raster data are saved as delimited ascii files and can be easily loaded into numpy arrays.

Define the paths to the resampled data

In [9]:
ppt_file = os.path.join(input_ws, "ppt_50m.txt")
tmax_file = os.path.join(input_ws, "tmax_50m.txt")
tmin_file = os.path.join(input_ws, "tmin_50m.txt")

Load the data into numpy arrays and reshape to appropriate shape

In [10]:
nhru = ml.modelgrid.nnodes

ppt = np.genfromtxt(ppt_file).reshape(12, nhru)
tmax = np.genfromtxt(tmax_file).reshape(12, nhru)
tmin = np.genfromtxt(tmin_file).reshape(12, nhru)

### Building the PRMS Data File for temp_1sta

pyGSFLOW includes the `PrmsData` class to allow reading, writing, and editing of climate station data files. This section will show how to go from a pandas dataframe to a dataframe commpatible with `PrmsData`

**1st)** add date columns to the dataframe using:

In [11]:
climate_df = bu.add_prms_date_columns_to_df(climate_df, "date")

climate_df.head()

,tmax,tmin,precip,runoff,date,Year,Month,Day,Hour,Minute,Second
0,85,30,0.0,2.3,10/1/1980,1980,10,1,0,0,0
1,81,32,0.0,2.3,10/2/1980,1980,10,2,0,0,0
2,83,30,0.0,2.3,10/3/1980,1980,10,3,0,0,0
3,80,32,0.0,2.3,10/4/1980,1980,10,4,0,0,0
4,80,32,0.0,2.3,10/5/1980,1980,10,5,0,0,0


**2nd)** rename the observation columns to have the zero based station number in their name. This is important because PRMS can use multiple climate stations with other climate modeules

In [12]:
climate_df.rename(
    columns={
        'precip': 'precip_0',
        'tmin': 'tmin_0',
        'tmax': 'tmax_0',
        'runoff': 'runoff_0',
        'date': 'Date'
    },
    inplace=True
)

**3rd)** reorder the datafame so Year, Month, Day, Hour, Minute, and Second are the first entries

In [13]:
cdfcols = [
        "Year", "Month", "Day", "Hour", "Minute", "Second",
        "tmax_0", "tmin_0", "precip_0", "runoff_0", "Date"
    ]
climate_df = climate_df[cdfcols]

**4th)** perform any necessary unit conversions before building the `PrmsData` object

In [14]:
climate_df["tmax_0"] = bu.fahrenheit_to_celsius(climate_df["tmax_0"].values)
climate_df["tmin_0"] = bu.fahrenheit_to_celsius(climate_df["tmin_0"].values)
climate_df.head()

,Year,Month,Day,Hour,Minute,Second,tmax_0,tmin_0,precip_0,runoff_0,Date
0,1980,10,1,0,0,0,29.444444,-1.111111,0.0,2.3,10/1/1980
1,1980,10,2,0,0,0,27.222222,0.000000,0.0,2.3,10/2/1980
2,1980,10,3,0,0,0,28.333333,-1.111111,0.0,2.3,10/3/1980
3,1980,10,4,0,0,0,26.666667,0.000000,0.0,2.3,10/4/1980
4,1980,10,5,0,0,0,26.666667,0.000000,0.0,2.3,10/5/1980


#### Finally build the `PrmsData` object

In [15]:
prms_data = gsflow.prms.PrmsData(data_df=climate_df)
prms_data.data_df.head()

,Year,Month,Day,Hour,Minute,Second,tmax_0,tmin_0,precip_0,runoff_0,Date
0,1980,10,1,0,0,0,29.444444,-1.111111,0.0,2.3,10/1/1980
1,1980,10,2,0,0,0,27.222222,0.000000,0.0,2.3,10/2/1980
2,1980,10,3,0,0,0,28.333333,-1.111111,0.0,2.3,10/3/1980
3,1980,10,4,0,0,0,26.666667,0.000000,0.0,2.3,10/4/1980
4,1980,10,5,0,0,0,26.666667,0.000000,0.0,2.3,10/5/1980


### Building climate `ParameterRecord` objects

The `builder_utils` module contains a number of functions that can be used to build `ParameterRecord` objects that can then be added to an existing `PrmsParameter` object (and later written to file).

Here we show the included climate methods

In [16]:
# get mean montly values from each ppt, tmax, and tmin for use in calculations
mean_ppt = bu.get_mean_monthly_from_df(climate_df, "precip_0")
mean_tmax = bu.get_mean_monthly_from_df(climate_df, "tmax_0", temperature=True)
mean_tmin = bu.get_mean_monthly_from_df(climate_df, "precip_0", temperature=True)

# calculate rain adj and snow adj factors from PRISM and means
rain_adj = bu.rain_adj(ppt, mean_ppt)
snow_adj = bu.snow_adj(ppt, mean_ppt)

# set lapse rates (convert to celsius in line)
tmin_lapse = bu.tmin_lapse(lapse_df.tmin_lapse.values * (5 / 9))
tmax_lapse = bu.tmax_lapse(lapse_df.tmax_lapse.values * (5 / 9))

# tmax and tmin adj are set to zero b/c lapse rates are used in sagehen
tmax_adj = bu.tmax_adj(nhru)
tmin_adj = bu.tmin_adj(nhru)

# calculate the jensen haise coeficients
jh_coef = bu.calculate_jensen_haise(ml.modelgrid.top, mean_tmin, mean_tmax)

### Adding climate `ParameterRecord` objects to the `PrmsParameters` object

In this section `ParameterRecord` objects are added to the `PrmsParameters` object using the built in `add_record_object` method.

The `add_record_object` method has two parameters:
   - `record_object` : a `ParameterRecord` object
   - `replace` : bool, a flag to replace an existing parameter if it exists. Default is True

In [17]:
parameters.add_record_object(rain_adj)
parameters.add_record_object(snow_adj)
parameters.add_record_object(tmin_lapse)
parameters.add_record_object(tmax_lapse)
parameters.add_record_object(tmax_adj)
parameters.add_record_object(tmin_adj)
parameters.add_record_object(jh_coef)

### Adding the temperature station elevation to the `PrmsParameters` object

The `ParameterRecord` allows users to create a new record and add it to the object using the `add_record()` method. This method has a number of input parameters and the most important ones are described here:
   - `name` : name of the parameter
   - `values` : list of parameter values
   - `dimensions` : 2 dimensional list of parameter dimensions in the format [[dimension name, dimesnion value], ...] ex. [["nmonths", 12], ["nhru", 6391]]
   - `datatype` : prms data type, 1=int, 2=float, 3=double, 4=string
   - `replace` : bool, if true replace existing `ParameterRecord`, default is False 

In [18]:
parameters.add_record(
    "tsta_elev",
    values=[1932.4,],
    dimensions=[["ntemp", 1]],
    datatype=2
)

### Adding the PRMS outlet station and runoff observation location

Adding the PRMS outlet station and runoff observation location can be accomplished using the `add_record` method described above. 

In this example the outlet station and runoff observation location are in the same location as the pour point used for watershed delineation. We'll load up the shapefile and then programatically get the hru number of these parameters.

In [19]:
with shapefile.Reader(shp_file) as r:
    pour_point = r.shape(0).points

# get the zero based node number of the outlet station
outlet_sta = ml.modelgrid.intersect(*pour_point[0])
outlet_sta = ml.modelgrid.get_node([(0,) + outlet_sta])

structuredgrid.py:777: UserWarning: The interface 'intersect(self, x, y, local=False, forgive=False)' has been deprecated. Use the intersect(self, x, y, z=None, local=False, forgive=False) interface instead.


Now add the outlet_sta and id_obsrunoff parameters to the `PrmsParameters` object

In [20]:
parameters.add_record(name="nobs", values=[1,])

parameters.add_record(
    "outlet_sta",
    values=[outlet_sta[0] + 1,],
    dimensions=[["one", 1]],
    datatype=1
)

parameters.add_record(
    "id_obsrunoff",
    values=[outlet_sta[0] + 1, ],
    dimensions=[["one", 1]],
    datatype=1
)

## Save the Sagehen 50m parameter and data files for use in the next notebook

The `PrmsParameters` and `PrmsData` objects can be saved using the built in `write()` method. This parameter file will be used in the next notebook, where we assemble, run, and calibrate the GSFLOW model. 

In [21]:
parameters.write(os.path.join(output_ws, "sagehen_50m_ncal.param"))
prms_data.write(os.path.join(output_ws, "sagehen_50m.data"))

## Defining climate by HRU with pyGSFLOW

An alternative method for defining climate information is to use the `climate_hru` module in GSFLOW or PRMS. When `climate_hru` is specified daily prms climate input values are stored in a `.day` file and their path is specified in the control file.

Here is a short example on how to define the climate using climate_hru with pygsflow.

In [22]:
# define path to GSFLOW control file
control_file = os.path.join(input_ws, "sagehen_50m_initial.control")

# load the control file
control = gsflow.ControlFile.load_from_file(control_file)

### Update the climate module specified in the control file

In this example we update the `precip_module` and `temp_module`. Other climate related modules can be updated in the same fashion.

In [23]:
# change precip and temp modules to climate_hru
control.precip_module = ["climate_hru",]
control.temp_module = ["climate_hru",]

print(control.temp_module)


####
temp_module
1
4
climate_hru
####


### Creating climate by hru files

The `PrmsDay` class allows users to craft climate by hru files and load, edit, and write climate by hru files. Here is a quick example using the previously loaded Prism precipitation data.

In this example, we are assuming that our model is only 12 days long and the prism data is not monthly normals, but daily data for those 12 days.

In [24]:
import datetime
# get the date header 
date_header = gsflow.prms.PrmsDay.date_header
cbh_dict = {"date": []}

# set to the day before our model begins
dt = datetime.datetime(2000, 9, 30)
# now set the dates
for _ in range(1, 13):
    dt += datetime.timedelta(days=1)
    cbh_dict["date"].append(dt)

for hru in range(nhru):
    cbh_dict[hru] = ppt[:, hru]
    
cbh_df = pd.DataFrame(cbh_dict)
cbh_df.set_index("date", inplace=True)

print(cbh_df.head())

                 0           1           2           3           4      \
date                                                                     
2000-10-01  170.172175  169.398904  168.625632  167.852360  167.079089   
2000-10-02  169.933959  169.165424  168.396890  167.628355  166.859820   
2000-10-03  160.387253  159.727663  159.068073  158.408483  157.748893   
2000-10-04   78.308347   77.937106   77.565865   77.194624   76.823382   
2000-10-05   47.942778   47.750940   47.559102   47.367264   47.175426   

                 5           6           7           8           9      ...  \
date                                                                    ...   
2000-10-01  166.305817  165.478152  164.597119  163.716087  162.835054  ...   
2000-10-02  166.091286  165.252816  164.345731  163.438646  162.531561  ...   
2000-10-03  157.089303  156.388469  155.647170  154.905871  154.164572  ...   
2000-10-04   76.452141   76.063864   75.658873   75.253883   74.848892  ...   
2000-10

### Adding climate by hru data to the `PrmsDay` object

The `PrmsDay` object has 4 input parameters that must be supplied to create a new object:
   - `f` : file name for the cbh file
   - `variable_name` : prms cbh variable name
   - `dataframe` : pandas dataframe (formatted as shown in block 31)
   - `nhru` : number of hru's

In [25]:
ppt_day = gsflow.prms.PrmsDay(
    os.path.join(output_ws, "ppt_day.cbh"),
    variable_name="hru_ppt",
    dataframe=cbh_df,
    nhru=nhru
)

### Writing the `PrmsDay` object to file

The `PrmsDay` object can be written to file using the `write()` method

In [26]:
ppt_day.write()

### Add the climate by hru file name to the `ControlFile` object

For this example the `add_record` method can be used to add the `precip_day` parameter to the control file. The `add_record` method takes the following parameters:
   - `name` : name of the parameter record
   - `values` : list of parameter record values

In [27]:
control.add_record("precip_day", ["ppt_day.cbh",])
print(control.precip_day)


####
precip_day
1
4
ppt_day.cbh
####


### Write the `ControlFile` object to file

The control file can be written from the `ControlFile` object using the `write()` method

In [28]:
# change the model workspace for this control file
control.model_dir = output_ws

# write
control.write("sagehen_cbh_ex.control")